In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'e:\\MyOnlineCourses\\ML_Projects\\arabic-digits-recognition'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTFEvaluationConfig:
    root_dir: Path
    data_path: str
    model_path:str
    

In [4]:
from src.ard.constants import *
from src.ard.utils.help import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_tf_evaluation_config(self) -> DataTFEvaluationConfig:
        config = self.config.data_tf_evaluation

        create_directories([config.root_dir])

        data_tf_evaluation_config = DataTFEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path
            
           
        )

        return data_tf_evaluation_config

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Any, Tuple
from dataclasses import dataclass
import os
from ard import logger
from ard.utils.tf_utils import build_dataset, finalize_dataset

tf.get_logger().setLevel('ERROR')  # Stop tf WARNINGS


class ModelEvaluation:
    def __init__(self, config: DataTFEvaluationConfig):
        self.config = config
        
        
    def predict(self):
        # Prepare to collect test audio and labels
        test_files = pd.read_csv(os.path.join(self.config.data_path))
        logger.info(f"Total Test files: {len(test_files)}")
        test = self._create_dataset(test_files['path'].tolist())
        model = tf.keras.models.load_model(self.config.model_path)
        test_audio = []
        test_labels = []

        for audio, label in test:
            # Convert tensors to numpy arrays
            audio_np = audio.numpy()  
            test_audio.append(audio_np)
            test_labels.append(label.numpy()) 

        # Concatenate the batches into a single array
        test_audio = np.concatenate(test_audio, axis=0)  
        test_labels = np.concatenate(test_labels, axis=0)  

        # Now you can evaluate the model
        loss, accuracy = model.evaluate(test_audio, test_labels)
        logger.info(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
        
    def _create_dataset(self, files):
        AUTOTUNE = tf.data.AUTOTUNE

        test_ds = build_dataset(files, AUTOTUNE)

        return finalize_dataset(test_ds, AUTOTUNE)

In [7]:
try:
    config = ConfigurationManager()
    data_tf_evaluation_config = config.get_data_tf_evaluation_config()
    data_tf_evaluation = ModelEvaluation(config=data_tf_evaluation_config)
    data_tf_evaluation.predict()

except Exception as e:
    raise e

[2024-08-12 00:03:11,000: INFO: help: yaml file: config\config.yaml loaded successfully. Content size: 8]
[2024-08-12 00:03:11,004: INFO: help: Total directories created: 1]
[2024-08-12 00:03:11,011: INFO: help: Total directories created: 1]
[2024-08-12 00:03:11,027: INFO: 1423599473: Total Test files: 41]
2/2 [==============================] - 2s 161ms/step - loss: 1.7690 - accuracy: 0.7317
[2024-08-12 00:04:05,989: INFO: 1423599473: Test Loss: 1.7690, Test Accuracy: 0.7317]
